In [124]:
# !pip install folium

In [27]:
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import requests
from dotenv import load_dotenv
import os
import time

In [28]:
# 사용자 좌표 (위도, 경도)
points = {
    'A': np.array([37.531132809078464, 126.8821229873256]),  # 목동운동장
    'B': np.array([37.5703884202892, 126.992329841579]),  # 종로 3가
    'C': np.array([37.513992512175925, 127.10199193245755])   # 롯데월드
}

In [29]:
np.sum(list(points.values()), axis=0) / len(points)

array([ 37.53850458, 126.99214825])

In [30]:
# 최초의 중간 지점 -> 사용자들 좌표의 평균 좌표
mid_point = np.sum(list(points.values()), axis=0) / len(points)
print(mid_point)

[ 37.53850458 126.99214825]


In [31]:
# .env 파일에서 환경변수 로드
load_dotenv()

# 환경변수에서 API 키 불러오기
api_key = os.getenv('ODsay_APIKEY')

# 요청 URL
url = 'https://api.odsay.com/v1/api/searchPubTransPathT'

# 소요 시간 차이 limit
standard_time_gap = 20

# 유저별 중앙지점까지의 벡터값
unit_vecs = {}

# 소요 시간가중치
vecs_weight = {}

# 모든 유저의 출발지-중간지점 이동 소요시간을 0으로 기본 설정
defult_time = {key : 0 for key in points.keys()}




# 시도 횟수
trial = 1

while True : 
    end_lat, end_lon = mid_point 
    print(f'중간 지점 좌표: {mid_point}')
    for name, value in points.items():
        start_lat, start_lon = points[name]    

        params = {
            'SX': float(start_lon),
            'SY': float(start_lat),
            'EX': float(end_lon),
            'EY': float(end_lat),
            'apiKey': api_key
        }

        response = requests.get(url, params=params)
        data = response.json()

        short_time = data['result']['path'][0]['info']['totalTime']
        
        for i in range(len(data['result']['path'])):
            print(f'{name}유저의 최단 시간: {short_time}')
            time_cost = data['result']['path'][i]['info']['totalTime']
            
            if time_cost < short_time:
                short_time = time_cost

        defult_time[name] = short_time

    time_gap = max(defult_time.values()) - min(defult_time.values())
    print(f'유저별 소요 시간: {defult_time}')
    print(f'{trial}번쨰 시도입니다. / {time_gap}분 차이')
    trial += 1

    if time_gap > standard_time_gap : 
        for name, point in points.items():
            vec = point - mid_point
            norm = np.linalg.norm(vec)

            if norm == 0:
                unit_vec = np.array([0, 0])
            else:
                unit_vec = vec / norm

            unit_vecs[name] = unit_vec

        
        # weight = {'A': 18, 'B': 46, 'C': 35}
        for name, vec in unit_vecs.items():
            vecs_weight[name] = vec * defult_time[name] # defult_time = weight

        avg_vec = sum(vecs_weight.values()) / len(points)

        mid_point += (avg_vec * 0.01)
        time.sleep(10)
        
    else :
        print(f"중간 지점은 {mid_point}입니다")
        for name, value in defult_time.items():
            print(f'{name}의 소요 시간: {value}분')
        break
            

중간 지점 좌표: [ 37.53850458 126.99214825]
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
A유저의 최단 시간: 40
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
B유저의 최단 시간: 25
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
C유저의 최단 시간: 42
유저별 소요 시간: {'A': 40, 'B': 25, 'C': 42}
1번쨰 시도입니다. / 17분 차이
중간 지점은 [ 37.53850458 126.99214825]입니다
A의 소요 시간: 40분
B의 소요 시간: 25분
C의 소요 시간: 42분


In [16]:
data

{'error': {'msg': '검색결과가 없습니다.', 'code': '-99'}}

In [9]:
data['result']['path'][0]['subPath']


[{'trafficType': 3, 'distance': 452, 'sectionTime': 7},
 {'trafficType': 1,
  'distance': 2450,
  'sectionTime': 6,
  'stationCount': 4,
  'lane': [{'name': '수도권 신림선', 'subwayCode': 117, 'subwayCityCode': 1000}],
  'intervalTime': 6,
  'startName': '신림',
  'startX': 126.929643,
  'startY': 37.484732,
  'endName': '보라매',
  'endX': 126.920428,
  'endY': 37.500408,
  'way': '보라매',
  'wayCode': 1,
  'door': '0-0',
  'startID': 11718,
  'endID': 11714,
  'startExitNo': '3',
  'startExitX': 126.92961510921224,
  'startExitY': 37.48387398515152,
  'passStopList': {'stations': [{'index': 0,
     'stationID': 11718,
     'stationName': '신림',
     'x': '126.929652',
     'y': '37.484736'},
    {'index': 1,
     'stationID': 11717,
     'stationName': '당곡',
     'x': '126.927559',
     'y': '37.490253'},
    {'index': 2,
     'stationID': 11716,
     'stationName': '보라매병원',
     'x': '126.923812',
     'y': '37.49297'},
    {'index': 3,
     'stationID': 11715,
     'stationName': '보라매공원',
     '